In [1]:
import numpy as np
import pandas as pd

/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_17310/1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import requests
base_url = "http://localhost:2000"
upstox_url = "https://api.upstox.com/v2"

In [3]:
import os
import sys

backend_path = os.path.abspath(os.path.join(os.getcwd(), '../backend'))
sys.path.append(backend_path)
from app.internal.firebase import db

/Users/aaryanpagar/Library/Python/3.11/lib/python/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### NSE Tickers Wrangling

In [4]:
instruments = pd.read_csv("data/upstox/NSE.csv")
instruments.head()

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
0,NSE_INDEX|Nifty 50,17.0,NaN,Nifty 50,21995.85,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
1,NSE_INDEX|NIFTY100 EQL Wgt,NaN,NaN,NIFTY100 EQL Wgt,29240.40,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
2,NSE_INDEX|NIFTY50 EQL Wgt,NaN,NaN,NIFTY50 EQL Wgt,27648.65,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
3,NSE_INDEX|NiftyM150Momntm50,NaN,NaN,NiftyM150Momntm50,54049.45,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
4,NSE_INDEX|Nifty Auto,48.0,NaN,Nifty Auto,21520.75,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX


In [5]:
equities = instruments.query('instrument_type == "EQUITY" and exchange == "NSE_EQ"')
equities.head()

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
77,NSE_EQ|DUMMYSAN005,14747.0,011NSETEST,011NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
78,NSE_EQ|DUMMYSAN006,14751.0,021NSETEST,021NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
79,NSE_EQ|DUMMYSAN007,14753.0,031NSETEST,031NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
80,NSE_EQ|DUMMYSAN008,14755.0,041NSETEST,041NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
81,NSE_EQ|DUMMYSAN009,14758.0,051NSETEST,051NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ


In [6]:
indexes = instruments.query('instrument_type == "INDEX" and exchange == "NSE_INDEX"')
indexes.head()

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
0,NSE_INDEX|Nifty 50,17.0,NaN,Nifty 50,21995.85,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
1,NSE_INDEX|NIFTY100 EQL Wgt,NaN,NaN,NIFTY100 EQL Wgt,29240.40,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
2,NSE_INDEX|NIFTY50 EQL Wgt,NaN,NaN,NIFTY50 EQL Wgt,27648.65,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
3,NSE_INDEX|NiftyM150Momntm50,NaN,NaN,NiftyM150Momntm50,54049.45,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX
4,NSE_INDEX|Nifty Auto,48.0,NaN,Nifty Auto,21520.75,NaN,NaN,NaN,NaN,INDEX,NaN,NSE_INDEX


GOAL ~ 2400 NSE listed companies
Remove:
- Government issued instruments
- Anything with %
- NSETEST

In [7]:
randoms = ["%", "NSETEST"]
goi_issues = ["GOI STRIPS", "GOI TBILL", "GOI LOAN"]

def remove_names(name):
    return all(i not in name for i in randoms + goi_issues)

equities = equities[equities["name"].apply(lambda x: remove_names(x))]
equities["exchange"] = equities["exchange"].str.replace("NSE_EQ", "NSE")

equities.shape


(2894, 12)

Remove Indexes with no exchange token

In [8]:
indexes.replace(np.nan, None, inplace=True)
indexes.dropna(subset=["exchange_token"], inplace=True)
indexes["exchange"] = indexes["exchange"].str.replace("NSE_INDEX", "NSE")

indexes.shape

/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_17310/3341430381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indexes.replace(np.nan, None, inplace=True)
/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_17310/3341430381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indexes.dropna(subset=["exchange_token"], inplace=True)
/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_17310/3341430381.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

(43, 12)

### POST Equities to BE

In [9]:
rename_cols = {
    "tradingsymbol": "ticker",
    "instrument_key": "upstox_instrument_key"
}
cols = ["exchange", "exchange_token", "name", "ticker", "instrument_type", "upstox_instrument_key", "lot_size"]


In [10]:
def post_tickers(tickers):
    s, f = 0, 0
    failed = []
    for p in tickers:
        resp = requests.post(base_url + "/tickers/", json=p)
        if resp.status_code == 201:
            s += 1
        else:
            f += 1
            failed.append(p)
            print(resp.json())
    return s, f, failed

In [11]:
q =  db.collection("tickers").where("smallcase_name", "!=", "")
smallcase_relevant = []
async for doc in q.stream():
    smallcase_relevant.append(doc.to_dict())

len(smallcase_relevant)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/Users/aaryanpagar/Library/Python/3.11/lib/python/site-packages/google/api_core/retry/retry_unary.py:146: UserWarning: Using the synchronous google.api_core.retry.Retry with asynchronous calls may lead to unexpected results. Please use google.api_core.retry_async.AsyncRetry instead.
  warnings.warn(_ASYNC_RETRY_WARNING)


347

In [12]:
# payload = equities[cols].to_dict(orient="records")

# s, f, failed = post_tickers(payload)

# print(f"Success: {s}, Failure: {f}")
# print(f"Failed {failed}")


04.05 - 43 Indexes + 6563 equities posted

05.05 - Deleted and posted only relevant equities 2894 (indexes have overlapping exchange tokens, do NOT post yet)

In [14]:
smallcase_relevant[0]

{'created_at': '2024-05-05T12:19:19.602296Z',
 'exchange': 'NSE',
 'lot_size': 1,
 'smallcase_name': '360 One Wam Ltd',
 'upstox_instrument_key': 'NSE_EQ|INE466L01038',
 'exchange_token': 13061,
 'updated_at': '2024-05-05T12:19:19.602299Z',
 'ticker': '360ONE',
 'instrument_type': 'EQUITY',
 'name': '360 ONE WAM LIMITED'}

### Fetch candle stick data and POST to Equities

In [16]:
import calendar
from datetime import datetime

equities.rename(columns=rename_cols, inplace=True)
# payload = equities[cols].to_dict(orient="records")
payload = [smallcase_relevant[0]]

# years = ["2023", "2022", "2021", "2020", "2019", "2018", "2017", "2016"]
# months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
years = ["2024"]
# months = ["01", "02", "03", "04"]
months = ["01"]

def get_last_day_of_month(year: int, month: int) -> int:
    return calendar.monthrange(year, month)[1]

def format_date_path(date: datetime) -> str:
    return date.strftime("%Y-%m-%d")


s, f, p = 0, 0, 0
failed = []
passive = []
for i, equity in enumerate(payload):
    exchange_token = int(equity["exchange_token"])
    instrument_key = equity["upstox_instrument_key"]

    for year in years:
        for month in months:
            f = True
            last_day = get_last_day_of_month(int(year), int(month))
            from_date = f"{year}-{month}-01"
            to_date = f"{year}-{month}-{last_day}"

            try:
                resp = requests.get(upstox_url + f"/historical-candle/{instrument_key}/day/{to_date}/{from_date}")
                if resp.status_code == 200:
                    candles = resp.json()["data"]["candles"]
                    if len(candles) > 0:
                        request = { "month": month, "year": year }

                        # Daily candles
                        daily = {}
                        running_high, running_low = 0, 0
                        for candle in candles:
                            timestamp = format_date_path(datetime.fromisoformat(candle[0]))
                            high = candle[2]
                            low = candle[3]
                            
                            daily[timestamp] = {
                                "open": candle[1],
                                "high": candle[2],
                                "low": candle[3],
                                "close": candle[4],
                                "volume": candle[5],
                                "open_interest": candle[6]
                            }
                        request["daily"] = daily

                        # Monthly candle
                        monthly_high = max(entry["high"] for entry in daily.values())
                        monthly_low = min(entry["low"] for entry in daily.values())
                        monthly_volume = sum(entry["volume"] for entry in daily.values())
                        monthly_open = candles[-1][1]
                        monthly_close = candles[0][4]
                        request["monthly"] = {
                            "open": monthly_open,
                            "high": monthly_high,
                            "low": monthly_low,
                            "close": monthly_close,
                            "volume": monthly_volume
                        }

                        internal_resp = requests.post(base_url + f"/tickers/{exchange_token}/candles?date={from_date}", json=request)
                        if internal_resp.status_code == 201:
                            s += 1
                        else:
                            failed.append(equity)
                            print("BE POST failed", internal_resp.status_code, internal_resp.json())
                    else:
                        f = False
                        passive.append(equity)
                        print("No candles", equity.get("ticker"), year, month)
                        break
                else:
                    failed.append(equity)
                    print("Upstox GET failed", resp.status_code, resp.json())

            except Exception as e:
                failed.append(equity)
                print("Exception", e)
        if not f:
            break

months_per_equity = len(months) * len(years)
print(f"Success: {s / months_per_equity}, Failure: {len(failed)}, Passive: {len(passive)}")


BE POST failed 422 {'detail': [{'type': 'missing', 'loc': ['query', 'date'], 'msg': 'Field required', 'input': None, 'url': 'https://errors.pydantic.dev/2.7/v/missing'}]}
Success: 0.0, Failure: 1, Passive: 0


In [45]:
# add 200 to this to get index of last successful equity
i, s, (s/months_per_equity)

(346, 1388, 347.0)

In [47]:
# # 1st iteration | 25m55s | s = 1869 | f = 0 | p = 1822 | x = 37
# # 2nd iteration | 191m 22.6 s | i = 200, 200 successful, stopped at 13285 -> Mahindra and Mahindra Finserv
# # 3rd iteration | 114m 10.4s | i = 146, s = 13260 (/96) | f = 0 | p = 18 | DONEEE 🥳
# # 4th iteration | 11m 3s | i = 346 | s = 1388 | s/month_per_equity = 347 | f = 0 | p = 0 | 2024 DONEEE 🥳

# last_stock_id = 5549
# x1 = next((i for (i, d) in enumerate(payload) if int(d["exchange_token"]) == last_stock_id), None) # no of stocks posted
# s , len(failed), len(passive), x1


No candles 360ONE 2019 01

No candles AUBANK 2017 01

No candles AAVAS 2018 01

No candles ACI 2022 01

No candles CAPACITE 2017 01

No candles CHOLAHLDNG 2017 01

No candles COCHINSHIP 2017 01

No candles CRAFTSMAN 2021 01

No candles CREDITACC 2018 01

No candles CROMPTON 2016 01

No candles DALBHARAT 2018 01

No candles DIVGIITTS 2023 01

No candles EMBASSY 2022 01

No candles ENDURANCE 2016 01

No candles ERIS 2017 01

No candles FIVESTAR 2022 01

No candles GATEWAY 2022 01

No candles GODREJAGRO 2017 01

No candles HGINFRA 2018 01

No candles HOMEFIRST 2021 01

No candles HDIL 2021 01

No candles ISEC 2018 01

No candles IEX 2017 01

No candles KPITTECH 2019 01

No candles KFINTECH 2022 01

No candles KIMS 2021 01

No candles LTIM 2016 01

No candles LAURUSLABS 2016 01

No candles MGL 2016 01

Exception ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

No candles MINDSPACE 2021 01

No candles NXST 2023 01

No candles ORIENTELEC 2018 01

No candles PARAGMILK 2016 01

No candles POLYCAB 2019 01

No candles PRINCEPIPE 2019 01

No candles RITES 2018 01

No candles RAILTEL 2021 01

No candles SBILIFE 2017 01

No candles SIS 2017 01

No candles SANDHAR 2018 01

No candles SONACOMS 2021 01

No candles STARCEMENT 2017 01

No candles TIINDIA 2017 01

No candles UNITECH 2021 01

No candles VBL 2016 01

No candles WESTLIFE 2019 01

No candles LIQUIDCASE 2023 01

Success: 138.125, Failure: 0, Passive: 18

We have understood, it is unfeasible to scrape all the data. 

Every stock took 45 seconds in the 1st iteration.
45 * 3000 = 135,000 = ~33 hours

Even after refactoring to optimse this by a factor of 2 to 22 seconds, it will take ~15 hours

Even if it is optimised to an unrealistic 10s, it will take ~8 hours

Instead, we deploy this code as a CF. Find only the equities we need from the Smallcases and invoke the CF to fetch data for that specific stock. This should reduce the equity universe by at least 80%.

22 * 0.2 * 3000 = 13,200 = ~3 hours which is much more manageable

### Delete Tickers

Delete tickers with no smallcase_name to keep only relevant equities and keep the db clean

In [18]:
# q = db.collection("tickers")

# deleted = []
# not_deleted = []
# async for doc in q.stream():
#     equity = doc.to_dict()
#     if not equity.get("smallcase_name"):
#         await doc.reference.delete()
#         deleted.append(equity["ticker"])
#     else:
#         not_deleted.append(equity["ticker"])

# len(deleted), len(not_deleted)

(2544, 347)

Originally posted equities = 2894

Relevant for Smallcases = 347

2894 - 347 = 2547 (Deleted 2-3 while testing) LETSGO 🥳

## External API

In [124]:
import requests

url = "https://api.upstox.com/v2/historical-candle/NSE_EQ|INE144J01027/month/2023-02-01/2023-01-01"

payload={}
headers = {
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{"status":"success","data":{"candles":[["2023-02-01T00:00:00+05:30",81.4,82.45,66.65,67.3,1564023,0],["2023-01-01T00:00:00+05:30",91.05,95.3,77,80.5,2362341,0]]}}


In [7]:
import requests

url = "https://api.upstox.com/v2/historical-candle/NSE_EQ|INF0R8F01034/day/2024-12-31/2024-01-01"

payload={}
headers = {
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{"status":"success","data":{"candles":[["2024-05-10T00:00:00+05:30",102.13,102.13,102.11,102.12,7833192,0],["2024-05-09T00:00:00+05:30",102.08,102.08,99,102.07,4728840,0],["2024-05-08T00:00:00+05:30",105.1,105.1,102.03,102.05,4532466,0],["2024-05-07T00:00:00+05:30",105.05,105.05,102.02,102.03,3811894,0],["2024-05-06T00:00:00+05:30",102.02,102.02,102,102.01,2527361,0],["2024-05-03T00:00:00+05:30",102.01,102.01,101.99,102,4006096,0],["2024-05-02T00:00:00+05:30",101.96,101.96,98.85,101.95,3897714,0],["2024-04-30T00:00:00+05:30",101.58,103.41,98.85,101.93,3091777,0],["2024-04-29T00:00:00+05:30",101.91,101.91,101.89,101.89,2873411,0],["2024-04-26T00:00:00+05:30",101.89,101.89,98.75,101.88,2508108,0],["2024-04-25T00:00:00+05:30",101.83,101.83,101.81,101.82,3442963,0],["2024-04-24T00:00:00+05:30",102.95,102.95,98.75,101.81,1247486,0],["2024-04-23T00:00:00+05:30",101.8,101.8,101.78,101.79,1947511,0],["2024-04-22T00:00:00+05:30",101.78,101.78,101.76,101.77,1833835,0],["2024-04-19T00:00:00+05:30